<a href="https://colab.research.google.com/github/bdipesh3045/356-Days-Of-Data-Science/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import numpy as np

import pandas as pd

In [29]:

import kagglehub
from kagglehub import KaggleDatasetAdapter
import os



# Load the latest version
path = kagglehub.dataset_download(
  "uciml/iris",

  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)
print("Path to dataset files:", path)
print("Files inside the path:", os.listdir(path))
csv_file = os.path.join(path,os.listdir(path)[0])


Path to dataset files: /kaggle/input/iris
Files inside the path: ['Iris.csv', 'database.sqlite']


In [30]:
df=pd.read_csv(csv_file)

In [31]:
df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [32]:
df['Species'].value_counts()

,count
Species,
Iris-setosa,50
Iris-versicolor,50
Iris-virginica,50


In [33]:
df.sample(5)

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
41,42,4.5,2.3,1.3,0.3,Iris-setosa
121,122,5.6,2.8,4.9,2.0,Iris-virginica
53,54,5.5,2.3,4.0,1.3,Iris-versicolor
107,108,7.3,2.9,6.3,1.8,Iris-virginica
34,35,4.9,3.1,1.5,0.1,Iris-setosa


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             150 non-null    int64  
 1   SepalLengthCm  150 non-null    float64
 2   SepalWidthCm   150 non-null    float64
 3   PetalLengthCm  150 non-null    float64
 4   PetalWidthCm   150 non-null    float64
 5   Species        150 non-null    object 
dtypes: float64(4), int64(1), object(1)
memory usage: 7.2+ KB


In [35]:
df.isnull().sum()

,0
Id,0
SepalLengthCm,0
SepalWidthCm,0
PetalLengthCm,0
PetalWidthCm,0
Species,0


In [59]:

X_df=df[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']]
X=X_df.values

In [62]:
y_df=df['Species']
y=y_df.values

In [74]:
from sklearn.model_selection import train_test_split, GridSearchCV

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=43,stratify=y)


In [76]:
from sklearn.neighbors import KNeighborsClassifier



# Define parameter grid
param_grid = {
    "n_neighbors": range(1, 21),               # try k = 1 to 20
    "weights": ["uniform", "distance"],        # uniform or distance-based
    "metric": ["euclidean", "manhattan"]       # different distance metrics
}

# Initialize KNN
knn = KNeighborsClassifier()

# Grid search with cross-validation
grid_search = GridSearchCV(
    knn,
    param_grid,
    cv=5,                # 5-fold cross-validation
    scoring="accuracy",  # optimize for accuracy
    n_jobs=-1            # use all CPU cores
)




grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(1, 21),
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [93]:
results = pd.DataFrame(grid_search.cv_results_)
results=(results[["params", "mean_test_score", "std_test_score", "rank_test_score"]]).sort_values(by="mean_test_score",ascending=False)
pd.set_option('display.max_colwidth', None)   # show full column content
pd.set_option('display.max_rows', None)       # show all rows (optional)
pd.set_option('display.max_columns', None)    # show all columns (optional)
results[:10]

,params,mean_test_score,std_test_score,rank_test_score
4,"{'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}",0.960000,0.053333,1
14,"{'metric': 'euclidean', 'n_neighbors': 8, 'weights': 'uniform'}",0.960000,0.032660,1
53,"{'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'distance'}",0.960000,0.032660,1
51,"{'metric': 'manhattan', 'n_neighbors': 6, 'weights': 'distance'}",0.960000,0.032660,1
57,"{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}",0.960000,0.032660,1
55,"{'metric': 'manhattan', 'n_neighbors': 8, 'weights': 'distance'}",0.960000,0.032660,1
52,"{'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}",0.960000,0.032660,1
54,"{'metric': 'manhattan', 'n_neighbors': 8, 'weights': 'uniform'}",0.946667,0.049889,8
56,"{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}",0.946667,0.049889,8
15,"{'metric': 'euclidean', 'n_neighbors': 8, 'weights': 'distance'}",0.946667,0.026667,10


In [95]:
print("Best Parameters:", grid_search.best_params_)
print("Best CV Score:", grid_search.best_score_)

Best Parameters: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}
Best CV Score: 0.9600000000000002


In [98]:
best_knn = grid_search.best_estimator_
print(grid_search.best_estimator_)
y_pred = best_knn.predict(X_test)

print("\nTest Accuracy:", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("\nClassification Report:\n", classification_report(y_test, y_pred))

KNeighborsClassifier(metric='euclidean', n_neighbors=3)

Test Accuracy: 97.33 %

Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        25
Iris-versicolor       0.96      0.96      0.96        25
 Iris-virginica       0.96      0.96      0.96        25

       accuracy                           0.97        75
      macro avg       0.97      0.97      0.97        75
   weighted avg       0.97      0.97      0.97        75



In [94]:
y_pred=knn.predict(X_test)


NotFittedError: This KNeighborsClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [72]:
from sklearn.metrics import accuracy_score,confusion_matrix
knn_accuracy = (accuracy_score(y_test, y_pred))
print("Accuracy:", round(knn_accuracy*100,2))

Accuracy: 97.33


In [69]:
cm = confusion_matrix(
    y_test,
    y_pred,
    labels=["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
)

cm

array([[25,  0,  0],
       [ 0, 24,  1],
       [ 0,  1, 24]])

In [63]:
y_df.value_counts()


,count
Species,
Iris-setosa,50
Iris-versicolor,50
Iris-virginica,50
